In [2]:
import numpy as np
import pandas as pd

In [ ]:
# Sliding window parameters
WINDOW_SIZE = 200
STEP_SIZE = 50

file_path = "emg_data_txt/grip_1.txt"

data = []
gesture_repetition = 0

# Read text file
with open(file_path, "r") as file:
    for line in file:
        line = line.strip()
        
        if line.startswith("#"):
            gesture_repetition += 1
            continue
        
        values = line.split(";")
        values = [float(v) for v in values if v]
        
        data.append([gesture_repetition] + values + [1])  # 1 = Grip pattern 1

# Convert to df
columns = ["gesture_repetition", "ch1_env", "ch1_raw", "ch2_env", "ch2_raw", "ch3_env", "ch3_raw", "grip_pattern"]
df = pd.DataFrame(data, columns=columns)

windowed_data = []

# Perform sliding window segmentation
for repetition in df["gesture_repetition"].unique():
    subset = df[df["gesture_repetition"] == repetition]  # Get one repetition

    for start in range(0, len(subset) - WINDOW_SIZE, STEP_SIZE):
        window = subset.iloc[start : start + WINDOW_SIZE].drop(columns=["gesture_repetition"])  # Extract window
        flattened_window = window.values.flatten()  # Flatten for ML model input
        label = window["grip_pattern"].iloc[0]  # Keep the same label

        # Store as [features, label]
        windowed_data.append(np.append(flattened_window, label))

# Convert to DataFrame
num_features = WINDOW_SIZE * 6  # 6 channels per time step
columns = [f"feature_{i}" for i in range(num_features)] + ["grip_pattern"]
windowed_df = pd.DataFrame(windowed_data, columns=columns)

# Save processed dataset
windowed_df.to_csv("processed_emg_windows.csv", index=False)

windowed_df.head()

ValueError: 1201 columns passed, passed data had 1401 columns